In [1]:
import pandas as pd

from transformers import pipeline
from tqdm.auto import tqdm

from torch.utils.data import DataLoader, Dataset

import re

In [2]:
submit_df = pd.read_csv('test.csv', sep=';')
submit_df

,id,Текст инцидента
0,0,'Может создать петицию по нашей проблеме)..<br...
1,1,"'Очень надеялась, что недоразумение с Декабрис..."
2,2,'Хоть бы отремонтировали поликлинику на Гайве....
3,3,"'Добрый день! Получаю выплаты с 3 до 7 лет, по..."
4,4,'Добрый день! Подскажите пожалуйста куда обращ...
...,...,...
9738,9738,"'Вдоль Октябрьского 7, до арки, тоже нет освещ..."
9739,9739,'Добрый день !Как сейчас будет происходить вып...
9740,9740,'У нас в посёлке кто-то снял металлические вер...
9741,9741,"'Соликамский р-н, +79523287940. Здравствуйте, ..."


In [3]:
class SubmitDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return 'Spell correct: ' + self.original_list[i][:1000]

submit_dataset = SubmitDataset(submit_df['Текст инцидента'].values)
submit_dataset[0]

"Spell correct: 'Может создать петицию по нашей проблеме)..<br>Хоккейная коробка обязательно должна остаться в ЖК. Дети и Взрослые активно пользуются.<br>Благодарю за такое развёрнутое, грамотное письмо!!"

In [4]:
import torch

In [7]:
checkpoint = 'ruBert-base-topic-ner'

classifier = pipeline("text-classification", 
                      model=checkpoint,
                      tokenizer="ai-forever/ruBert-base",
                      batch_size=64, device=torch.device('cuda:0')
)

In [8]:
predictions = []

for prediction in tqdm(classifier(submit_dataset), total=len(submit_dataset)):
    predictions.append(prediction)

  0%|          | 0/9743 [00:00<?, ?it/s]

In [11]:
predictions = [i['label'] for i in predictions]

In [21]:
submit_example = pd.read_csv('submission.csv', sep=';')
submit_example

,id,Группа тем,Тема
0,0,Физическая культура и спорт,Строительство спортивной инфраструктуры
1,1,Физическая культура и спорт,Строительство спортивной инфраструктуры
2,2,Физическая культура и спорт,Строительство спортивной инфраструктуры
3,3,Физическая культура и спорт,Строительство спортивной инфраструктуры
4,4,Физическая культура и спорт,Строительство спортивной инфраструктуры
...,...,...,...
9738,9738,Физическая культура и спорт,Строительство спортивной инфраструктуры
9739,9739,Физическая культура и спорт,Строительство спортивной инфраструктуры
9740,9740,Физическая культура и спорт,Строительство спортивной инфраструктуры
9741,9741,Физическая культура и спорт,Строительство спортивной инфраструктуры


In [ ]:
with open("topic2big_topic.json", "r", encoding="UTF8") as f:
    topic2big_topic = json.load(f)

In [22]:
submit_example['Группа тем'] = [topic2big_topic[i] for i in predictions]
submit_example['Тема'] = predictions
submit_example

,id,Группа тем,Тема
0,0,Благоустройство,Отсутствие детских площадок
1,1,Общественный транспорт,Содержание остановок
2,2,Здравоохранение/Медицина,Содержание больниц
3,3,Социальное обслуживание и защита,Дети и многодетные семьи
4,4,Социальное обслуживание и защита,Улучшение жилищных условий
...,...,...,...
9738,9738,Благоустройство,Отсутствие фонарей освещения
9739,9739,Социальное обслуживание и защита,Дети и многодетные семьи
9740,9740,Дороги,Ливневые канализации (строительство и реконстр...
9741,9741,Мобилизация,Поддержка семей мобилизованных


In [24]:
submit_example.to_csv('rubert_base_ner.csv', sep = ';', index = False, encoding = 'utf-8')

In [26]:
pd.read_csv('rubert_base_ner.csv', sep=';')

,id,Группа тем,Тема
0,0,Благоустройство,Отсутствие детских площадок
1,1,Общественный транспорт,Содержание остановок
2,2,Здравоохранение/Медицина,Содержание больниц
3,3,Социальное обслуживание и защита,Дети и многодетные семьи
4,4,Социальное обслуживание и защита,Улучшение жилищных условий
...,...,...,...
9738,9738,Благоустройство,Отсутствие фонарей освещения
9739,9739,Социальное обслуживание и защита,Дети и многодетные семьи
9740,9740,Дороги,Ливневые канализации (строительство и реконстр...
9741,9741,Мобилизация,Поддержка семей мобилизованных
